In [526]:
from pathlib import Path
import configparser
from psycopg2 import connect
from psycopg2.extras import execute_values
from psycopg2.extras import RealDictCursor
import json
import os
import subprocess
import shlex
from subprocess import Popen, PIPE
CONFIG = configparser.ConfigParser()
CONFIG.read(str(Path.home().joinpath('db.cfg')))
dbset = CONFIG['DBSETTINGS']
con = connect(**dbset)

In [678]:
def get_json(id, input, filename):
    CONFIG.read('db.cfg')
    password = CONFIG.get('DBSETTINGS', 'password')
    sql_json = """\COPY (SELECT jsonb_build_object('type', 'FeatureCollection', 'features', jsonb_agg(feature))FROM (SELECT jsonb_build_object('type', 'Feature', 'id', {}, 'geometry', ST_AsGeoJSON(geom)::jsonb, 'properties', to_jsonb(row) - 'id' - 'geom')AS feature FROM (select {}, geom from {} ) row) features )  TO {}""".format(id, id, input, filename)
    base = "psql -h 10.160.12.47 -d bigdata -U natalie -W -c"
    psqlcomm = subprocess.run(shlex.split(base) + [sql_json, ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, env={'PGPASSWORD':password})

In [697]:
def match(id, input, filename, flags):
    get_json(id, input, filename)
    command = "node bin/run match "
    subprocess.run(shlex.split(command)+ [filename, ]  + shlex.split(flags), stdout=subprocess.PIPE, stderr=subprocess.PIPE)

In [707]:
def get_command(id, input, filename, output, flags):
    
    """
    This function takes street networks from postgres database, send to matching in sharedstreets API then send referenced streets back to postgres database
    
    Parameters
    ----------       
    id :  string
        Unique id column name for the input network. E.g. 'analysis_id' for bluetooth segments
        
    input : string
        Input table name in postgres 
        
    output : string
        Output table name in postgres for the matched and unmatched result
    
    flags : string
        Additional flags for matching (e.g. --follow-line-direction)
    """
    match(id, input, filename, flags)
    match_exists = os.path.isfile(filename+'.matched.geojson')
    if match_exists:
        command=['ogr2ogr -f "PostgreSQL"  PG:"dbname=bigdata host=10.160.12.47 user=natalie password='+password+'" "{}.matched.geojson"'.format(filename) +' '+'-nln "{}_matched"'.format(output)]
        subprocess.call(command, shell=True)
        print('Finished importing matched geojson in {}_matched'.format(output))
    else:
        print('No match')
        
    unmatch_exists = os.path.isfile(filename+'.unmatched.geojson')
    if unmatch_exists:
        command=['ogr2ogr -f "PostgreSQL"  PG:"dbname=bigdata host=10.160.12.47 user=natalie password='+password+'" "{}.unmatched.geojson"'.format(filename) +' '+'-nln "{}_unmatched"'.format(output)]
        subprocess.call(command, shell=True)
    else: 
        print('All matched')

In [708]:
get_command('analysis_id', 'bluetooth.segments', 'bluetooth_segments.json', 'bluetooth_test_test_testtt', '--snap-intersections --search-radius=10')

Finished importing matched geojson in bluetooth_test_test_testtt_matched
